In [1]:
import os
import sys
import math
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


In [3]:
## Folder Locations
TEST_IMAGE_FOLDER = '../data/raw/Test/testset/'
TEST_CSV = '../data/raw/Test/testset.csv'

TRAIN_IMAGE_FOLDER = '../data/raw/Train/testset/'
TRAIN_CSV = '../data/raw/Train/testset.csv'


# VALID_IMAGE_FOLDER = '../data/raw/Train/testset/'
# VALID_CSV = '../data/raw/Valid/new_valid_set.csv'

In [4]:
train_set = pd.read_csv(TRAIN_CSV)
test_set = pd.read_csv(TEST_CSV)
train_set.head()

,Data,Label
0,600795.jpeg,10
1,627152.jpeg,10
2,119963.jpeg,10
3,118264.jpeg,10
4,199420.jpeg,10


In [5]:
test_set.head()

,Data,Label
0,632755.jpeg,10
1,496855.jpeg,10
2,155390.jpeg,10
3,265013.jpeg,10
4,496360.jpeg,10


In [6]:
test_set['Label'] = test_set['Label'].apply(str)
train_set['Label'] = train_set['Label'].apply(str)

In [7]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2609 entries, 0 to 2608
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Data    2609 non-null   object
 1   Label   2609 non-null   object
dtypes: object(2)
memory usage: 40.9+ KB


In [8]:
CLASSES = len(train_set['Label'].unique())

# Image Data generator

In [35]:
from keras_preprocessing.image import ImageDataGenerator

In [36]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [54]:
TARGET_SIZE=(224, 224)


train_generator=datagen.flow_from_dataframe(
dataframe=train_set,
directory=TRAIN_IMAGE_FOLDER,
x_col="Data",
y_col="Label",
subset="training",
batch_size=32,
seed=RANDOM_STATE,
shuffle=True,
class_mode="categorical",
target_size=TARGET_SIZE)


valid_generator=datagen.flow_from_dataframe(
dataframe=train_set,
directory=TRAIN_IMAGE_FOLDER,
x_col="Data",
y_col="Label",
subset="validation",
batch_size=24,
seed=RANDOM_STATE,
shuffle=True,
class_mode="categorical",
target_size=TARGET_SIZE)


test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_set,
directory=TEST_IMAGE_FOLDER,
x_col="Data",
y_col=None,
batch_size=32,
seed=RANDOM_STATE,
shuffle=False,
class_mode=None,
target_size=TARGET_SIZE)

Found 7200 validated image filenames belonging to 48 classes.
Found 2400 validated image filenames belonging to 48 classes.
Found 2609 validated image filenames.


In [55]:
TARGET_SIZE[1]

224

## MobileNet

In [56]:
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from tensorflow.keras.optimizers import Adam

In [57]:
base_model = keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False)

In [58]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(48,activation='softmax')(x) #final layer with softmax activation

In [59]:
model=Model(inputs=base_model.input,outputs=preds)

In [60]:

for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [61]:

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                
                   epochs=10,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID)

Epoch 1/10
225/225 [==============================] - 34s 152ms/step - loss: 1.1969 - accuracy: 0.6621 - val_loss: 39.6735 - val_accuracy: 0.0000e+00
Epoch 2/10
225/225 [==============================] - 34s 151ms/step - loss: 0.1234 - accuracy: 0.9668 - val_loss: 29.7051 - val_accuracy: 0.0000e+00
Epoch 3/10
225/225 [==============================] - 34s 150ms/step - loss: 0.0497 - accuracy: 0.9896 - val_loss: 27.2032 - val_accuracy: 0.0000e+00
Epoch 4/10
225/225 [==============================] - 34s 149ms/step - loss: 0.0898 - accuracy: 0.9793 - val_loss: 22.4973 - val_accuracy: 0.0000e+00
Epoch 5/10
225/225 [==============================] - 34s 150ms/step - loss: 0.0592 - accuracy: 0.9876 - val_loss: 21.9891 - val_accuracy: 0.0000e+00
Epoch 6/10
225/225 [==============================] - 34s 149ms/step - loss: 0.0228 - accuracy: 0.9958 - val_loss: 25.4819 - val_accuracy: 0.0000e+00
Epoch 7/10
225/225 [==============================] - 34s 151ms/step - loss: 0.0208 - accuracy: 0.99

In [64]:
predictions = model.predict(test_generator)

In [66]:
label_map = (train_generator.class_indices)

In [69]:
label_map

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '12': 4,
 '13': 5,
 '14': 6,
 '15': 7,
 '16': 8,
 '17': 9,
 '18': 10,
 '19': 11,
 '2': 12,
 '20': 13,
 '21': 14,
 '22': 15,
 '23': 16,
 '24': 17,
 '25': 18,
 '26': 19,
 '27': 20,
 '28': 21,
 '29': 22,
 '3': 23,
 '30': 24,
 '31': 25,
 '32': 26,
 '33': 27,
 '34': 28,
 '35': 29,
 '36': 30,
 '37': 31,
 '38': 32,
 '39': 33,
 '4': 34,
 '40': 35,
 '41': 36,
 '42': 37,
 '43': 38,
 '44': 39,
 '45': 40,
 '46': 41,
 '47': 42,
 '5': 43,
 '6': 44,
 '7': 45,
 '8': 46,
 '9': 47}

In [74]:
model.predict_proba(test_generator)

AttributeError: 'Model' object has no attribute 'predict_proba'

In [ ]:
# # at this point, the top layers are well trained and we can start fine-tuning
# # convolutional layers from inception V3. We will freeze the bottom N layers
# # and train the remaining top layers.

# # let's visualize layer names and layer indices to see how many layers
# # we should freeze:
# for i, layer in enumerate(base_model.layers):
#    print(i, layer.name)

# # we chose to train the top 2 inception blocks, i.e. we will freeze
# # the first 249 layers and unfreeze the rest:
# for layer in model.layers[:249]:
#    layer.trainable = False
# for layer in model.layers[249:]:
#    layer.trainable = True

# # we need to recompile the model for these modifications to take effect
# # we use SGD with a low learning rate
# from tensorflow.keras.optimizers import SGD
# model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# # we train our model again (this time fine-tuning the top 2 inception blocks
# # alongside the top Dense layers
# model.fit(...)